## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тестовое множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации документа по категориям, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [1]:
!pip install sklearn
!pip install numpy
!pip install pymorphy2
!pip install pandas
!pip install tqdm
!pip install gensim
!pip install nltk


import re
import pandas as pd
import pymorphy2
from gensim.models import Word2Vec
from nltk.stem.snowball import SnowballStemmer
from scipy.stats import uniform
from sklearn import svm, metrics
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
import numpy as np
import random
from tqdm.notebook import tqdm
from gensim.test.utils import common_texts
from gensim.models import FastText
import warnings
warnings.filterwarnings("ignore")

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1321 sha256=bf850f9e8427d6ad162beee27a6ff171bebec2c8798b59d3a45a51d8b4a05640
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\22\0b\40\fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


In [2]:
train = pd.read_csv("news_train.txt",sep='\t', header=None)
train = train.rename({0:'tag', 1:'header', 2:'content'}, axis='columns')

test = pd.read_csv("news_test.txt",sep='\t', header=None)
test= test.rename({0:'tag', 1:'header', 2:'content'}, axis='columns')

train

,tag,header,content
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,Нападающий «Вашингтон Кэпиталз» Александр Овеч...
1,culture,Рекордно дорогую статую майя признали подделкой,"Власти Мексики объявили подделкой статую майя,..."
2,science,Samsung представила флагман в защищенном корпусе,Южнокорейская Samsung анонсировала защищенную ...
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,Контрольно-дисциплинарный комитет (КДК) РФС сн...
4,media,Hopes & Fears объединится с The Village,Интернет-издание Hopes & Fears объявило о свое...
...,...,...,...
14995,life,Составлен рейтинг лучших европейских пляжей 20...,Опубликован рейтинг лучших европейских пляжей ...
14996,media,В «Снобе» объяснили причину смены формата,Генеральный директор «Сноб медиа» Марина Гевор...
14997,economics,Минфин предложил штрафовать за биткоины на 50 ...,"Минфин разработал законопроект, устанавливающи..."
14998,life,Мэл Гибсон заплатит бывшей подруге 750 тысяч д...,Актер и режиссер Мэл Гибсон выплатит своей быв...


In [3]:
morph = pymorphy2.MorphAnalyzer()

In [4]:
def process_sentences(df):
    processed_texts = []
    for i, text in tqdm(enumerate(df['content']), desc = 'Processing dataframe', total = df['content'].shape[0]):
        text = text.lower()
        words = re.findall(r'\b\w+\b', text)
        tokens = [morph.parse(word)[0].normal_form for word in words]
        processed_texts.append(tokens)
    return processed_texts

In [5]:
processed_train = process_sentences(train)
processed_test = process_sentences(test)

In [6]:
model = FastText(sentences=processed_train, window=5, min_count=1, workers=5)

In [7]:
model.train(processed_train, total_examples=10, epochs=5)

(12882705, 14796365)

In [8]:
wv = model.wv

In [9]:
wv.most_similar(positive=['закон'], topn=10)

[('законов12', 0.9540018439292908),
 ('законно', 0.9444084167480469),
 ('законодательство', 0.9119880199432373),
 ('законцовка', 0.911150336265564),
 ('законник', 0.8937592506408691),
 ('законодательно', 0.8933196663856506),
 ('законопроект', 0.8912474513053894),
 ('закономерно', 0.8767651319503784),
 ('законодатель', 0.8725363612174988),
 ('беззаконие', 0.8713452219963074)]

In [10]:
wv.most_similar(positive=['соревнование', 'победа'], negative=['кино'], topn=5)

[('матч', 0.7345455288887024),
 ('водапобеда', 0.7244129180908203),
 ('побег', 0.7235247492790222),
 ('матчбол', 0.7231491208076477),
 ('матчтво', 0.7160069346427917)]

In [11]:
def print_metrics(y_pred, y_test):
    print(f'Precision: {precision_score(y_pred, y_test, average="weighted"):.2f}')
    print(f'Recall: {recall_score(y_pred, y_test, average="weighted"):.2f}')
    print(f'F1 score: {f1_score(y_pred, y_test, average="weighted"):.2f}')
    print(f'Accuracy: {accuracy_score(y_pred, y_test):.2f}')

In [12]:
X_train = [' '.join(sentence) for sentence in processed_train]
y_train = train['tag']
X_test = [' '.join(sentence) for sentence in processed_test]
y_test = test['tag']

In [13]:
tfidf = TfidfVectorizer()
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

In [14]:
log_reg = RandomizedSearchCV(
    LogisticRegression(),
    param_distributions={
        'C': np.arange(0.01, 1.01, 0.01)
    }, 
    verbose = 1
)

naive_bayes = RandomizedSearchCV(
    MultinomialNB(),
    param_distributions={
        'alpha': np.arange(0.5, 1.51, 0.01)
    }, 
    verbose = 1
)

In [15]:
log_reg.fit(X_train_vec, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 15.3min finished


RandomizedSearchCV(estimator=LogisticRegression(),
                   param_distributions={'C': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  ])},
                   verbose=1)

In [16]:
naive_bayes.fit(X_train_vec, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    4.1s finished


RandomizedSearchCV(estimator=MultinomialNB(),
                   param_distributions={'alpha': array([0.5 , 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 ,
       0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71,
       0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8 , 0.81, 0.82,
       0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9 , 0.91, 0.92, 0.93,
       0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.  , 1.01, 1.02, 1.03, 1.04,
       1.05, 1.06, 1.07, 1.08, 1.09, 1.1 , 1.11, 1.12, 1.13, 1.14, 1.15,
       1.16, 1.17, 1.18, 1.19, 1.2 , 1.21, 1.22, 1.23, 1.24, 1.25, 1.26,
       1.27, 1.28, 1.29, 1.3 , 1.31, 1.32, 1.33, 1.34, 1.35, 1.36, 1.37,
       1.38, 1.39, 1.4 , 1.41, 1.42, 1.43, 1.44, 1.45, 1.46, 1.47, 1.48,
       1.49, 1.5 ])},
                   verbose=1)

In [17]:
print('logistic Regression:')
y_pred_lreg = log_reg.best_estimator_.predict(X_test_vec)
print_metrics(y_pred_lreg, y_test)

print('\nNaive Bayes:')
y_pred_nb = naive_bayes.predict(X_test_vec)
print_metrics(y_pred_nb, y_test)

logistic Regression:
Precision: 0.89
Recall: 0.88
F1 score: 0.88
Accuracy: 0.88

Naive Bayes:
Precision: 0.88
Recall: 0.82
F1 score: 0.85
Accuracy: 0.82


Почти идентичны, но логистическая регрессия чуть лучше